#Imports

In [ ]:
!pip install kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from sklearn.neighbors import KNeighborsClassifier # models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier # tree models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)
path = 'PATH_TO_DATASET_IN_DRIVE'

Mounted at /content/gdrive/


#Functions

In [ ]:
def verify_unnamed_in_base(base):
  if base.columns[0] == 'Unnamed: 0': # verifying if the first collumn is bugged
    base.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
  return base

In [ ]:
def return_bases_rus_sbc(type_undersampling, base_type, model_type, model_type_classe):
  if base_type == 'base_total':
    try:
        best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
    except:
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'.csv', sep=',')

    if type_undersampling == 'random_under':
      try:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_NORMALIZADO_MODELO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_NORMALIZADO_MODELO.csv', sep=',')
    elif type_undersampling == 'sbc':
      try:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_undersampling+'.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_MODELO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_NORMALIZADO_MODELO.csv', sep=',')

    best_base = verify_unnamed_in_base(best_base)
    best_results = verify_unnamed_in_base(best_results)
    best_grid = verify_unnamed_in_base(best_grid)
  elif base_type == 'base_reduzida':
    try:
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO.csv', sep=',')
    except:
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_RECORTE_FEITO.csv', sep=',')

    if type_undersampling == 'random_under':
      try:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')
    elif type_undersampling == 'sbc':
      try:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_undersampling+'_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')

    #xgboost_base_moda_sbc_sbc_RECORTE_FEITO_NORMALIZADO_sbc_NORMALIZADO
    best_base = verify_unnamed_in_base(best_base)
    best_results = verify_unnamed_in_base(best_results)
    best_grid = verify_unnamed_in_base(best_grid)

  return best_base, best_results, best_grid

In [ ]:
def get_best_auc_roc_near_miss(type_near_miss, base_type, model_type, type_undersampling):
  neighbors = [3, 5, 7, 9, 11]
  if base_type == 'base_total':
    if (type_near_miss == '1') or (type_near_miss == '2'):
      try: # first loop to get results and compare with others
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

      auc_roc_max = best_results['AUC ROC'][0]
      k_of_auc_roc_max = 1
      base_with_bigger_auc_roc = best_results.copy()
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_NORMALIZADO.csv', sep=',')
      best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_NORMALIZADO_MODELO.csv', sep=',')

      for n in neighbors:
        try:
          best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        except:
          best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

        if ((best_results['AUC ROC'][0]) > auc_roc_max):
          auc_roc_max = best_results['AUC ROC'][0]
          k_of_auc_roc_max = n # pegando o k
          base_with_bigger_auc_roc = best_results.copy() # pegando a base
          best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_NORMALIZADO.csv', sep=',')
          best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_NORMALIZADO_MODELO.csv', sep=',')

        #print(best_results['AUC ROC'][0])
    elif type_near_miss == '3':
      try: # aqui executo o primeiro loop para pegar os resultados para comparar com os outros
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

      auc_roc_max = best_results['AUC ROC'][0]
      k_1_of_auc_roc_max = 1
      k_2_of_auc_roc_max = 3
      base_with_bigger_auc_roc = best_results.copy()
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_NORMALIZADO.csv', sep=',')
      best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_NORMALIZADO_MODELO.csv', sep=',')

      k_of_auc_roc_max = {
          'k1': k_1_of_auc_roc_max,
          'k2': k_2_of_auc_roc_max
      }

      neighbors_2 = [1, 3, 5, 7, 9, 11]

      for n in neighbors_2:
        for n2 in neighbors_2:
          if n2 > n:
            try:
              best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
            except:
              best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

            if ((best_results['AUC ROC'][0]) > auc_roc_max):
              auc_roc_max = best_results['AUC ROC'][0]
              k_of_auc_roc_max = {
                  'k1': n,
                  'k2': n2
              }
              base_with_bigger_auc_roc = best_results.copy() # pegando a base
              best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_NORMALIZADO.csv', sep=',')
              best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_NORMALIZADO_MODELO.csv', sep=',')
  elif base_type == 'base_reduzida':
    if (type_near_miss == '1') or (type_near_miss == '2'):
      best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

      auc_roc_max = best_results['AUC ROC'][0]
      k_of_auc_roc_max = 1
      base_with_bigger_auc_roc = best_results.copy()
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_RECORTE_FEITO_NORMALIZADO.csv', sep=',')
      best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k1_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')

      for n in neighbors:
        try:
          best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_RECORTE_FEITO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
        except:
          best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

        if ((best_results['AUC ROC'][0]) > auc_roc_max):
          auc_roc_max = best_results['AUC ROC'][0]
          k_of_auc_roc_max = n # pegando o k
          base_with_bigger_auc_roc = best_results.copy() # pegando a base
          best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_RECORTE_FEITO_NORMALIZADO.csv', sep=',')
          best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+'_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')

        #print(best_results['AUC ROC'][0])
    elif type_near_miss == '3':
      try: # aqui executo o primeiro loop para pegar os resultados para comparar com os outros
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_RECORTE_FEITO'+type_undersampling+'_NORMALIZADO.csv', sep=',')
      except:
        best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

      auc_roc_max = best_results['AUC ROC'][0]
      k_1_of_auc_roc_max = 1
      k_2_of_auc_roc_max = 3
      base_with_bigger_auc_roc = best_results.copy()
      best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_RECORTE_FEITO_NORMALIZADO.csv', sep=',')
      best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k13_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')

      k_of_auc_roc_max = {
          'k1': k_1_of_auc_roc_max,
          'k2': k_2_of_auc_roc_max
      }

      neighbors_2 = [1, 3, 5, 7, 9, 11]

      for n in neighbors_2:
        for n2 in neighbors_2:
          if n2 > n:
            try:
              best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_RECORTE_FEITO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')
            except:
              best_results = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_RECORTE_FEITO_NORMALIZADO_'+type_undersampling+'_NORMALIZADO.csv', sep=',')

            if ((best_results['AUC ROC'][0]) > auc_roc_max):
              auc_roc_max = best_results['AUC ROC'][0]
              k_of_auc_roc_max = {
                  'k1': n,
                  'k2': n2
              }
              base_with_bigger_auc_roc = best_results.copy() # pegando a base
              best_base = pd.read_csv(path+'bases-pre-proc/base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_RECORTE_FEITO_NORMALIZADO.csv', sep=',')
              best_grid = pd.read_csv(path+'resultados/'+base_type+'/'+model_type+'/best_grid/'+model_type+'_base_moda_'+type_undersampling+'_'+type_near_miss+'_k'+str(n)+str(n2)+'_RECORTE_FEITO_NORMALIZADO_MODELO.csv', sep=',')


  #print(best_base.iloc[1])
  best_base = verify_unnamed_in_base(best_base)
  base_with_bigger_auc_roc = verify_unnamed_in_base(base_with_bigger_auc_roc)
  best_grid = verify_unnamed_in_base(best_grid)

  #print(f'AUC ROC maxima: {auc_roc_max}\nK: {k_of_auc_roc_max}\nNearMiss:{type_near_miss}') # resultado final
  return best_base, base_with_bigger_auc_roc, best_grid, k_of_auc_roc_max

In [ ]:
def return_best_results_undersampling(base_t, model_name, model_class):
  best_base, best_results_random_under, best_grid = return_bases_rus_sbc('random_under', base_t, model_name, model_class)
  best_base, best_results_sbc, best_grid = return_bases_rus_sbc('sbc', base_t, model_name, model_class)
  best_base, best_results_nm1, best_grid, k_of_auc_roc_max = get_best_auc_roc_near_miss('1', base_t, model_name, 'near_miss')
  best_base, best_results_nm2, best_grid, k_of_auc_roc_max = get_best_auc_roc_near_miss('2', base_t, model_name, 'near_miss')
  best_base, best_results_nm3, best_grid, k_of_auc_roc_max = get_best_auc_roc_near_miss('3', base_t, model_name, 'near_miss')

  return best_results_random_under, best_results_sbc, best_results_nm1, best_results_nm2, best_results_nm3

In [ ]:
def filter_model_name(model_name):
  if model_name == 'decision_tree':
    return 'Decision Tree'
  elif model_name == 'random_forest':
    return 'Random Forest'
  elif model_name == 'adaboost':
    return 'AdaBoost'
  elif model_name == 'gbm':
    return 'GBM'
  elif model_name == 'xgboost':
    return 'XGBoost'
  elif model_name == 'svm':
    return 'SVM'
  elif model_name == 'knn':
    return 'KNN'

In [ ]:
def create_figure(best_results_random_under, best_results_sbc, best_results_nm1, best_results_nm2, best_results_nm3, base_t, model_name):
  titulo = f'{filter_model_name(model_name)}'

  figure = go.Figure(
      #layout_title_text=f'{base_t} {model_name}'
      layout_title_text=titulo
  )
  categories = ['Acurácia', 'Precisão', 'Sensibilidade', 'Especificidade', 'F1-score', 'AUC ROC', 'Acurácia']

  figure.add_trace(go.Scatterpolar(
      r=best_results_random_under,
      theta=categories,
      name='Random Undersampling'
  ))

  figure.add_trace(go.Scatterpolar(
      r=best_results_sbc,
      theta=categories,
      name='SBC'
  ))

  figure.add_trace(go.Scatterpolar(
      r=best_results_nm1,
      theta=categories,
      name='NearMiss-1'
  ))

  figure.add_trace(go.Scatterpolar(
      r=best_results_nm2,
      theta=categories,
      name='NearMiss-2'
  ))

  figure.add_trace(go.Scatterpolar(
      r=best_results_nm3,
      theta=categories,
      name='NearMiss-3'
  ))

  figure.update_layout(
      polar=dict(
          radialaxis=dict(
              visible=True,
              range=[0.0, 1.0]
          )
      ), showlegend=True
  )

  #figure.write_image(file=path+f'teste-restantes/images/grafico_radar/{titulo}.png')
  #figure.show()
  return figure, titulo

In [ ]:
def generate_graphics_results_undersampling():
  #base_type = ['base_total', 'base_reduzida']
  #base_type = ['base_reduzida']
  base_type = ['base_total']
  #cols_models = {'xgboost': XGBClassifier}
  cols_models = {'decision_tree': DecisionTreeClassifier, 'random_forest': RandomForestClassifier,
                  'adaboost': AdaBoostClassifier, 'gbm': GradientBoostingClassifier,
                  'knn': KNeighborsClassifier, 'svm': SVC, 'xgboost': XGBClassifier} # generalize

  for base_t in base_type:
    for model_name in cols_models.keys():
      best_results_random_under, best_results_sbc, best_results_nm1, best_results_nm2, best_results_nm3 = return_best_results_undersampling(base_t, model_name, cols_models[model_name])

      best_results_random_under['accuracy_2'] = best_results_random_under['acuracia'].iloc[0] # so it can appear right
      best_results_sbc['accuracy_2'] = best_results_sbc['acuracia'].iloc[0]
      best_results_nm1['accuracy_2'] = best_results_nm1['acuracia'].iloc[0]
      best_results_nm2['accuracy_2'] = best_results_nm2['acuracia'].iloc[0]
      best_results_nm3['accuracy_2'] = best_results_nm3['acuracia'].iloc[0]

      figure, titulo = create_figure(best_results_random_under.iloc[0], best_results_sbc.iloc[0], best_results_nm1.iloc[0],
                    best_results_nm2.iloc[0], best_results_nm3.iloc[0], base_t, model_name)
      #figure.write_image(file=path+f'teste-restantes/images/grafico_radar/{titulo}.png')

In [ ]:
def generate_graphics_results_undersampling_all_in_one():
  base_type = ['base_total']
  #cols_models = {'xgboost': XGBClassifier, 'decision_tree': DecisionTreeClassifier,}
  cols_models = {'decision_tree': DecisionTreeClassifier, 'random_forest': RandomForestClassifier, 'adaboost': AdaBoostClassifier,
                  'gbm': GradientBoostingClassifier, 'knn': KNeighborsClassifier, 'svm': SVC, 'xgboost': XGBClassifier} # generalize

  easier_names = []
  for prov_name in cols_models.keys():
    easier_names.append(filter_model_name(prov_name))

  easier_names_bold = []
  for names_f in easier_names:
    easier_names_bold.append('<b>'+names_f+'</b>') # turning into bold

  number_cols_ms = 4
  number_lines_ms = 2 # coloquei o bold em baixo
  fig = make_subplots(rows=number_lines_ms, cols=number_cols_ms, specs=[ [{'type': 'polar'}]*number_cols_ms,
   [{'type': 'polar'}]*number_cols_ms ], subplot_titles=easier_names_bold, horizontal_spacing=0.09, vertical_spacing=0.08) # criando a figura
  #categories = ['Acurácia', 'Precisão', 'Sensibilidade', 'Especificidade', 'F1-score', 'AUC ROC', 'Acurácia'] # portugues
  categories = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC ROC', 'Accuracy'] # engrish
  count_row=1
  count_col=1
  show_subtitle = None

  for base_t in base_type:
    for model_name in cols_models.keys():
      if count_col > 4:
        count_row = 2
        count_col = 1

      best_results_random_under, best_results_sbc, best_results_nm1, best_results_nm2, best_results_nm3 = return_best_results_undersampling(base_t, model_name, cols_models[model_name])
      best_results_random_under=best_results_random_under*100
      best_results_sbc=best_results_sbc*100
      best_results_nm1=best_results_nm1*100
      best_results_nm2=best_results_nm2*100
      best_results_nm3=best_results_nm3*100

      best_results_random_under['accuracy_2'] = best_results_random_under['acuracia'].iloc[0] # para poder aparecer direito no grafico
      best_results_sbc['accuracy_2'] = best_results_sbc['acuracia'].iloc[0]
      best_results_nm1['accuracy_2'] = best_results_nm1['acuracia'].iloc[0]
      best_results_nm2['accuracy_2'] = best_results_nm2['acuracia'].iloc[0]
      best_results_nm3['accuracy_2'] = best_results_nm3['acuracia'].iloc[0]

      if count_col>1:
        show_subtitle=False

      fig.add_trace(go.Scatterpolar(r=best_results_random_under.iloc[0], theta=categories, name='Random Undersampling', marker=dict(color='blue'),
                                    showlegend=show_subtitle, legendgroup='Random Undersampling'), row=count_row, col=count_col)
      fig.add_trace(go.Scatterpolar(r=best_results_sbc.iloc[0], theta=categories, name='SBC', showlegend=show_subtitle, marker=dict(color='red'),
                                    legendgroup='SBC'), row=count_row, col=count_col)
      fig.add_trace(go.Scatterpolar(r=best_results_nm1.iloc[0], theta=categories, name='NearMiss-1', showlegend=show_subtitle, marker=dict(color='green'),
                                    legendgroup='NearMiss-1'), row=count_row, col=count_col)
      fig.add_trace(go.Scatterpolar(r=best_results_nm2.iloc[0], theta=categories, name='NearMiss-2', showlegend=show_subtitle, marker=dict(color='purple'),
                                    legendgroup='NearMiss-2'), row=count_row, col=count_col)
      fig.add_trace(go.Scatterpolar(r=best_results_nm3.iloc[0], theta=categories, name='NearMiss-3', showlegend=show_subtitle, marker=dict(color='orange'),
                                    legendgroup='NearMiss-3'), row=count_row, col=count_col)

      '''fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0.0, 1.0], linewidth=0.1)), legend=dict(x=0.8, y=0.0),
                        font=dict(size=10.5))'''
      fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 100], linewidth=0.1)), legend=dict(x=0.8, y=0.0),
                        #font=dict(size=10.5))
                        font=dict(size=12, color='black', family='Helvetica'))
      #, showlegend=True)
      #fig.update_polars(radialaxis_categoryorder="array", radialaxis_categoryarray=["0%", "20%", "40%", "60%", "80%", "100%"])

      fig.update_annotations(font_size=14)
      count_col = count_col+1
  fig.show()
  #fig.write_image(file=path+f'teste-restantes/images/grafico_radar/teste_todos.png')

generate_graphics_results_undersampling_all_in_one() # ver ecra inteiro, download plot as png

In [ ]:
def create_figure_all(vetor_com_figuras):
  fig = make_subplots(rows=1, cols=1, specs=[ [{'type': 'polar'}] ])

  '''fig = fig.add_trace(vetor_com_figuras[0].data[0], row=1, col=1)
  fig = fig.add_trace(vetor_com_figuras[0].data[1], row=1, col=1)
  fig = fig.add_trace(vetor_com_figuras[0].data[2], row=1, col=1)'''

  for data_len in range(0, len(vetor_com_figuras[0].data)): 
    fig = fig.add_trace(vetor_com_figuras[0].data[data_len], row=1, col=1)

  fig.show()

In [ ]:
generate_graphics_results_undersampling()